# Домашнее задание 1

### Описание

В вашем распоряжении датасет с русскоязычными отзывами о мобильных телефонах с выставленным рейтингом от 1 до 5.
Ключевая задача – обучить любую модель регрессии (или классификации, если решите таким путём пойти) из пакетов scikit, XGBoost, LightGBM, CatBoost.


Необходимая метрика:

1. Со звёздочкой (дополнительный балл) – MAE <= 0.5
2. Минимальное допустимое значение – МАЕ <= 1.0

### Что необходимо сделать

1. Откройте датасет
2. Разделите на обучение и тест
3. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 
4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).
5. Вывести ТОП-50 наиболее частотных токенов:
- только для униграмм
- только для биграмм
- только для триграмм
- для всех вариантов n-грамм одновременно

Напишите, какие наблюдения и выводы есть.
6. Повторите пункт 5, только отдельно для отзывов с рейтингом «4-5», «3» и «1-2». Есть ли ключевые отличия? Есть кандидаты на попадание в список стоп-слов?

7. Составьте список своих ключевых слов, для помощи можно использовать nltk и punctuations из string. 
8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer (экспериментируйте с параметрами min_df  и max_df). 
9. Обучение одну или несколько моделей машинного обучения на разных представлениях данных
10. Валидируйте модель. Если модель соответствует условиям метрик, то работа завершена. В ином случае, экспериментируйте, начиная с пункта 7. 
11. По всем попыткам обучить качественную модель пишите свои выводы и замечания, почему так получилось.


## Комментарии исполнителя

Я изменил порядок 2 и 3 пункта, потому что так намного удобнее

## 0. Импорт библиотк, определение констант

In [179]:
import os
import pandas as pd
import numpy as np
import spacy
import re

from string import punctuation
from collections import Counter

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from pymorphy3 import MorphAnalyzer

from nltk import ngrams
from nltk.corpus import stopwords

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from sklearn.metrics import classification_report

In [78]:
list(punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [79]:
tqdm.pandas()

In [80]:
#!python -m spacy download ru_core_news_sm

## 1. Откройте датасет

In [81]:
if os.path.exists("data/data_lemma.csv"):
    df = pd.read_csv("data/data_lemma.csv", engine='python')
else:    
    df = pd.read_csv("data/data.csv")

In [82]:
df.shape

(458433, 2)

In [83]:
df.head(10)

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5
5,6s gold 64gb,5
6,Мой первый айфон. Скажу честно- Эппл ранее ник...,5
7,Мне очень понравилась эта модель! Во-первых кл...,5
8,Долгое время пользовалась iPhone 5s 16Gb. Он м...,5
9,"Раньше был Samsung Galaxy Alpha SM-G850F 32Gb,...",4


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458433 entries, 0 to 458432
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  457837 non-null  object
 1   Rating  458433 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


In [85]:
df = df.dropna().reset_index(drop=True)
df.head()

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5


In [86]:
df['Rating'].value_counts()

Rating
5    228086
4    106503
3     53055
2     35705
1     34484
0         2
7         1
9         1
Name: count, dtype: int64

In [87]:
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]

In [88]:
df['Rating'].value_counts(normalize=True)

Rating
5    0.498186
4    0.232624
3    0.115883
2    0.077987
1    0.075320
Name: proportion, dtype: float64

Я закомментил все что ниже, потому что такую обработку лучше делать после лемматизации

In [89]:
df.duplicated().sum()

136735

In [90]:
df[df.duplicated()].head(10)

,Review,Rating
343,"Покупал 6-ку в интернет магазине, перед этими ...",4
354,Довольно долго сопротивлялся смартфонам. Да и ...,5
358,"Всем заядлым ""яблочникам"" хочу сказать,что чув...",4
359,"Отличный, как и все айфоны, но цены очень высока.",5
361,"Очень понравился телефон в использовании, мне ...",5
363,"Приятный телефон в использовании, Камера на 5,...",5
364,В целом доволен покупкой. Если любите пофоткат...,4
365,"iPhone 6 практически идеален. Отличный дизайн,...",5
373,"Если вы сомневаетесь нужен ли вам iPhone, если...",5
385,"Бесит, что защитные стекла все идут не на весь...",5


In [91]:
df["Review"].value_counts()

Review
Отличный телефон                                                                                                                                            114
Заключение: среди бюджетных смартфонов явлется безусловно одной из лучших моделей. Пока не собираюсь менять на другой аппарат.                               95
Рекомендую                                                                                                                                                   94
отличный телефон                                                                                                                                             93
СУПЕР                                                                                                                                                        89
                                                                                                                                                           ... 
Зборка Тайвань, Китай как-то подн

Основная причина дублей - короткие одинаковые комментарии, такие тубли можно исключить

In [92]:
# df = df.drop_duplicates()

In [93]:
df["Review"].value_counts()

Review
Отличный телефон                                                                                                                                            114
Заключение: среди бюджетных смартфонов явлется безусловно одной из лучших моделей. Пока не собираюсь менять на другой аппарат.                               95
Рекомендую                                                                                                                                                   94
отличный телефон                                                                                                                                             93
СУПЕР                                                                                                                                                        89
                                                                                                                                                           ... 
Зборка Тайвань, Китай как-то подн

In [94]:
df[df["Review"] == "-"].value_counts()

Review  Rating
-       5         19
        3          8
        4          5
        2          3
        1          1
Name: count, dtype: int64

Исключим записи с одинаковым комментарием, но выоским разрбросом оценки

In [95]:
#bad_comments = df["Review"].value_counts()
#bad_comments = bad_comments[bad_comments >= 3]
#bad_comments.head(20)

#bad_comments = bad_comments[bad_comments['value'].isin((4,5))]

In [96]:
df['Review'] = df['Review'].str.lower()

Уберём синтаксические знаки

In [97]:
punct = list(punctuation)
punct.remove(":")
punct.remove(")")
punct.remove("(")
punct = ''.join(punct)

In [98]:
df['Review']

0         3d touch просто восхитительная вещь! заряд дер...
1         отключается при температуре близкой к нулю, не...
2         в apple окончательно решили не заморачиваться,...
3         постарался наиболее ёмко и коротко описать все...
4         достойный телефон. пользоваться одно удовольст...
                                ...                        
457832    удобный, всё работает отлично, звонит, играет,...
457833    прошло больше года, притензий нет, при моей на...
457834    мой первый аппарат на андроиде. на данный моме...
457835    разбил iphone и не было желания покупать новый...
457836             очень доволен покупкой и всем советую...
Name: Review, Length: 457833, dtype: object

In [99]:
punct

'!"#$%&\'*+,-./;<=>?@[\\]^_`{|}~'

In [100]:
df['Review'] = df['Review'].str.replace(r'[' + punct + ']', '', regex=True)

## 2. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 

In [101]:
stop_words = set(stopwords.words('russian'))
stop_words.remove("не"); 

### spacy

In [102]:
spacy_nlp = spacy.load("ru_core_news_sm")

In [103]:
def spacy_lemmatize_text(text):
    try:
        doc = spacy_nlp(text)
    except:
        print(f'{text}')
        return ""
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

В этом месте у меня падала лемматизация, так я обнаружил что есть пустые комменты, но трай эксепт оставил, в пайморфек уже нет этого, потому что мусор удален

### pymorphy3

In [104]:
pymorphy3_analyzer = MorphAnalyzer()

In [105]:
def pymorphy3_lemmatize_text(text):
    lemmas = [pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split()]
    clean_tokens = [w for w in lemmas if not w in stop_words]  
    return (' '.join(clean_tokens))

### Лемматинизируем

In [106]:
if not "lemma" in df.columns:
    print('Лемматизируем текст');
    df["lemma"] = df["Review"].progress_apply(lambda row: pymorphy3_lemmatize_text(row))
    df.to_csv("data_lemma.csv", index=False)

Лемматизируем текст


100%|██████████| 457833/457833 [31:45<00:00, 240.29it/s]


In [107]:
df.shape

(457833, 3)

In [108]:
df = df.dropna()

In [109]:
df = df.drop_duplicates()

In [110]:
df.shape

(318702, 3)

In [111]:
if not os.path.exists("data/data_lemma_cleared.csv"):
    df.to_csv("data/data_lemma_cleared.csv")

In [112]:
df

,Review,Rating,lemma
0,3d touch просто восхитительная вещь заряд держ...,5,3d touch просто восхитительный вещь заряд держ...
1,отключается при температуре близкой к нулю неп...,4,отключаться температура близкий нуль непонятно...
2,в apple окончательно решили не заморачиваться ...,3,apple окончательно решить не заморачиваться де...
3,постарался наиболее ёмко и коротко описать все...,4,постараться наиболее ёмко коротко описать всё ...
4,достойный телефон пользоваться одно удовольствие,5,достойный телефон пользоваться удовольствие
...,...,...,...
457832,удобный всё работает отлично звонит играет сни...,5,удобный всё работать отлично звонить играть сн...
457833,прошло больше года притензий нет при моей нагр...,5,пройти большой год притензия нагрузка 5060 зво...
457834,мой первый аппарат на андроиде на данный момен...,5,первый аппарат андроид данный момент не разача...
457835,разбил iphone и не было желания покупать новый...,5,разбить iphone не желание покупать новый хотет...


## 3. Разделите на обучение и тест

In [151]:
X_train, X_test, y_train, y_test = train_test_split(df['lemma'], df['Rating'], test_size=0.2, stratify=df['Rating'])

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(254961,)
(254961,)
(63741,)
(63741,)


## 4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).

Я всё таки попробую разные способы, чтобы сравнить быстродействие и результат

In [114]:
def ngrams_stas(data, n):
    word_vectorizer = CountVectorizer(ngram_range=(n,n), analyzer='word')
    sparse_matrix = word_vectorizer.fit_transform(data)
    frequencies = sum(sparse_matrix).data
    return pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names_out(), columns=['frequency'])

### Униграммы

In [115]:
unigrams = list(df['lemma'].str.split()) 

In [116]:
unigrams = [x for xs in unigrams for x in xs]

In [117]:
len(unigrams)

14271484

In [118]:
print(len(set(unigrams)))

420176


In [119]:
unigrams

['3d',
 'touch',
 'просто',
 'восхитительный',
 'вещь',
 'заряд',
 'держать',
 'целый',
 'день',
 'розовый',
 'цвет',
 'смотреться',
 'очень',
 'необычно',
 'touch',
 'id',
 'очень',
 'быстрый',
 'удобный',
 'весь',
 'советовать',
 'телефон',
 'отключаться',
 'температура',
 'близкий',
 'нуль',
 'непонятно',
 'вести',
 'батарея',
 'apple',
 'окончательно',
 'решить',
 'не',
 'заморачиваться',
 'делать',
 'незначительный',
 'изменение',
 'телефон',
 'выдавать',
 'изменение',
 'инновация',
 'скопировать',
 'не',
 'функционал',
 'дизайн',
 'цена',
 'неадекватно',
 'завысить',
 'скачок',
 'курс',
 'говорить',
 'это',
 'не',
 'приходиться',
 'многие',
 'сказать',
 'hd',
 'разрешение',
 'достаточно',
 'диагональ',
 'мочь',
 'оно',
 'аппарат',
 'позиционироваться',
 'топовый',
 'решение',
 'итог',
 'получать',
 'банальный',
 'экономия',
 'производитель',
 'произойти',
 'время',
 'работа',
 'увеличить',
 'fhd',
 '1',
 'гб',
 'оперативка',
 'прошлый',
 'век',
 'не',
 'рассказывать',
 'сказка',


In [122]:
unigrams_counter = Counter(unigrams)
unigrams_counter.most_common(50)

[('не', 601265),
 ('телефон', 334158),
 ('это', 202917),
 ('всё', 156058),
 ('очень', 139791),
 ('хороший', 123638),
 ('экран', 86495),
 ('аппарат', 75303),
 ('купить', 73751),
 ('камера', 73240),
 ('работать', 70068),
 ('весь', 69983),
 ('день', 68988),
 ('год', 68721),
 ('свой', 60529),
 ('пользоваться', 58084),
 ('просто', 55686),
 ('который', 54841),
 ('мочь', 54581),
 ('большой', 53101),
 ('цена', 51346),
 ('ещё', 51293),
 ('смартфон', 50886),
 ('брать', 47484),
 ('модель', 47291),
 ('2', 45221),
 ('батарея', 45046),
 ('деньга', 43327),
 ('приложение', 42931),
 ('отличный', 42109),
 ('довольный', 41968),
 ('время', 41599),
 ('проблема', 41316),
 ('качество', 40578),
 ('покупать', 38474),
 ('покупка', 38110),
 ('рука', 35474),
 ('месяц', 35143),
 ('использование', 35127),
 ('первый', 34100),
 ('работа', 33717),
 ('интернет', 31403),
 ('хватать', 30193),
 ('память', 30132),
 ('целое', 30094),
 ('стоить', 29924),
 ('новый', 29672),
 ('стать', 29565),
 ('держать', 28740),
 ('хотеть', 

Попробуем создать нграммы через универсальную функцию с помощью CountVectorizer

In [120]:
ngrams_stas(df['lemma'], 1).sort_values('frequency', ascending=False)

,frequency
мпик,605356
мпxкакраз,337554
мпиксов,203177
мп3смерть,156611
мпг4,141172
...,...
дедушек,1
дедушекбабушка,1
дедушка,1
дедушкаа,1


Получился странный и непонятный мне результат

### биграммы и триграммы
попробуем через nltk

In [126]:

un, bi, th = [], [], []
for s in df['lemma']:
    sent = s.split()
    un.append(list(ngrams(sent, 1)))
    bi.append(list(ngrams(sent, 2)))
    th.append(list(ngrams(sent, 3)))

Работаем мнгновенно!

In [127]:
un = [x for xs in un for x in xs]
un_counter = Counter(un)
un_counter.most_common(50)

[(('не',), 601265),
 (('телефон',), 334158),
 (('это',), 202917),
 (('всё',), 156058),
 (('очень',), 139791),
 (('хороший',), 123638),
 (('экран',), 86495),
 (('аппарат',), 75303),
 (('купить',), 73751),
 (('камера',), 73240),
 (('работать',), 70068),
 (('весь',), 69983),
 (('день',), 68988),
 (('год',), 68721),
 (('свой',), 60529),
 (('пользоваться',), 58084),
 (('просто',), 55686),
 (('который',), 54841),
 (('мочь',), 54581),
 (('большой',), 53101),
 (('цена',), 51346),
 (('ещё',), 51293),
 (('смартфон',), 50886),
 (('брать',), 47484),
 (('модель',), 47291),
 (('2',), 45221),
 (('батарея',), 45046),
 (('деньга',), 43327),
 (('приложение',), 42931),
 (('отличный',), 42109),
 (('довольный',), 41968),
 (('время',), 41599),
 (('проблема',), 41316),
 (('качество',), 40578),
 (('покупать',), 38474),
 (('покупка',), 38110),
 (('рука',), 35474),
 (('месяц',), 35143),
 (('использование',), 35127),
 (('первый',), 34100),
 (('работа',), 33717),
 (('интернет',), 31403),
 (('хватать',), 30193),
 

Совпадает с тем, что делал вручную и работает достаточно быстро. Делаем универсальную функцию

In [136]:
def ngrams_stats_nltk(data):
    
    un, bi, th = [], [], []
    
    for s in data:
        sent = s.split()
        un.append(list(ngrams(sent, 1)))
        bi.append(list(ngrams(sent, 2)))
        th.append(list(ngrams(sent, 3)))
        
    un = [x for xs in un for x in xs]
    un = Counter(un)
    
    bi = [x for xs in bi for x in xs]
    bi = Counter(bi)
    
    th = [x for xs in th for x in xs]
    th = Counter(th)       
    
    return pd.DataFrame({
                        'Униграмы' : un.most_common(50),
                        'Биграммы' : bi.most_common(50),
                        'Триграммы' : th.most_common(50)  
                        })
      
        

## 5. Топы

### Топ 50 для всех отзывов

In [139]:
ngrams_stats_nltk(df["lemma"])


,Униграмы,Биграммы,Триграммы
0,"((не,), 601265)","((телефон, не), 22102)","((стоить, свой, деньга), 3026)"
1,"((телефон,), 334158)","((это, не), 18314)","((брать, не, пожалеть), 2765)"
2,"((это,), 202917)","((не, мочь), 14801)","((игра, не, играть), 1947)"
3,"((всё,), 156058)","((свой, деньга), 12979)","((телефон, очень, довольный), 1744)"
4,"((очень,), 139791)","((очень, довольный), 12333)","((телефон, свой, деньга), 1673)"
5,"((хороший,), 123638)","((это, телефон), 12167)","((очень, хороший, телефон), 1461)"
6,"((экран,), 86495)","((телефон, очень), 10116)","((удобно, лежать, рука), 1344)"
7,"((аппарат,), 75303)","((не, пожалеть), 9981)","((телефон, очень, хороший), 1336)"
8,"((купить,), 73751)","((очень, хороший), 9809)","((соотношение, цена, качество), 1309)"
9,"((камера,), 73240)","((не, очень), 9728)","((очень, довольный, покупка), 1208)"


## 6. Топы по оценкам

### Топ 50 для плохих отзывов

In [141]:
ngrams_stats_nltk(df.loc[df["Rating"] < 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 100462)","((телефон, не), 4441)","((не, советовать, покупать), 828)"
1,"((телефон,), 54283)","((не, советовать), 3955)","((не, советовать, брать), 590)"
2,"((это,), 33247)","((это, не), 3029)","((никто, не, советовать), 533)"
3,"((всё,), 19088)","((не, мочь), 2990)","((большой, не, купить), 445)"
4,"((купить,), 14602)","((это, телефон), 2229)","((не, покупать, телефон), 379)"
5,"((очень,), 13723)","((не, рекомендовать), 2150)","((стоить, свой, деньга), 351)"
6,"((хороший,), 12013)","((не, покупать), 2123)","((не, стоить, свой), 349)"
7,"((экран,), 11967)","((не, стоить), 1927)","((жить, свой, жизнь), 337)"
8,"((год,), 10559)","((не, работать), 1841)","((кой, случай, не), 288)"
9,"((работать,), 9892)","((большой, не), 1816)","((не, рекомендовать, покупка), 267)"


In [148]:
ngrams_stats_nltk(df.loc[df["Rating"] == 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 74793)","((телефон, не), 3152)","((телефон, не, плохой), 308)"
1,"((телефон,), 40482)","((это, не), 2237)","((стоить, свой, деньга), 279)"
2,"((это,), 24342)","((не, мочь), 2052)","((не, советовать, покупать), 219)"
3,"((всё,), 16079)","((это, телефон), 1570)","((не, советовать, брать), 200)"
4,"((хороший,), 13047)","((не, очень), 1433)","((оставлять, желать, хороший), 181)"
5,"((очень,), 12487)","((не, знать), 1273)","((деньга, не, стоить), 176)"
6,"((экран,), 10505)","((не, стоить), 1259)","((не, стоить, свой), 174)"
7,"((купить,), 8948)","((не, советовать), 1210)","((свой, деньга, не), 169)"
8,"((год,), 8787)","((не, плохой), 1175)","((игра, не, играть), 165)"
9,"((камера,), 8600)","((свой, деньга), 1156)","((жить, свой, жизнь), 161)"


### Топ 50 для хороших отзывов

In [142]:
ngrams_stats_nltk(df.loc[df["Rating"] > 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 426010)","((телефон, не), 14509)","((брать, не, пожалеть), 2714)"
1,"((телефон,), 239393)","((это, не), 13048)","((стоить, свой, деньга), 2396)"
2,"((это,), 145328)","((очень, довольный), 11750)","((телефон, очень, довольный), 1703)"
3,"((всё,), 120891)","((свой, деньга), 10907)","((игра, не, играть), 1684)"
4,"((очень,), 113581)","((не, мочь), 9759)","((телефон, свой, деньга), 1506)"
5,"((хороший,), 98578)","((не, пожалеть), 9709)","((очень, хороший, телефон), 1399)"
6,"((экран,), 64023)","((очень, хороший), 8996)","((телефон, очень, хороший), 1239)"
7,"((аппарат,), 57461)","((телефон, очень), 8730)","((удобно, лежать, рука), 1233)"
8,"((камера,), 57440)","((это, телефон), 8368)","((соотношение, цена, качество), 1211)"
9,"((весь,), 54667)","((хороший, телефон), 7999)","((очень, довольный, покупка), 1161)"


Смотреть объединенную статистику бессмысленно, так как частотность униграм превышает частотность би и триграмм, соотвественном будет состоять из униграм.

### Вывод

Чем выше размер энграм , тем существенней отличия в положительных и отрицательных отзывах. По триграммам отлично понятно - положительный или отрицательный отзыв. Гипотеза - фичи на триграммах дадут лучший результат. 
Заметил присутствие слова "это", думаю никакой смысловой нагрузке в данном случае оно не несёт и его можно смело удалить. Так как и слова "телефон" , "сей", "пора", "смартфон" . Остальное оставлю, чтобы не портить значащие би и три граммы.

## 7. Список стоп слов

Базовый список стоп слов и пунктуацию мы использовали в предыдущиз шагах, в этом сделаем дополнительный список на основании анализа энграм

In [149]:
custom_stopwords = ['это', 'телефон', 'сей', 'пора', 'смартфон']

## 8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer 

(экспериментируйте с параметрами min_df  и max_df). 

В интернетах пишут
So to sum it up, pruning the terms via min_df and max_df is to improve the performance, not the quality of clusters (as an example).
а так как перфоманс меня вполне себе устраивает, то я с этими параметрами экспериментирова не буду, качество гарантированно не улучшится

In [161]:
n_grams = (2, 2)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

preds = lr.predict(vec.transform(X_test))
preds = np.round(preds)
    
print(mean_absolute_error(y_test, preds))

0.5344283898903375


In [162]:
n_grams = (3, 3)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

preds = lr.predict(vec.transform(X_test))
preds = np.round(preds)
    
print(mean_absolute_error(y_test, preds))

0.5070519759652343


In [163]:
n_grams = (2, 3)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

preds = lr.predict(vec.transform(X_test))
preds = np.round(preds)
    
print(mean_absolute_error(y_test, preds))

0.4522677711363173


### CatBoost

In [164]:
text_features = ['lemma']

In [174]:
X_train = pd.DataFrame({'lemma': X_train})

In [177]:
model_cb = CatBoostRegressor(iterations=1689,
                                depth = 6,
                                loss_function='MAE',
                                thread_count=-1, 
                                random_seed=42, 
                                verbose=0, 
                                text_features=text_features,
                                task_type = "GPU" 
                                
)


model_cb.fit(X_train, y_train, text_features=text_features, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAE is/are not implemented for GPU


In [182]:
X_test = pd.DataFrame({'lemma': X_test})
 
y_pred = model_cb.predict(X_test)
print(classification_report(y_test, y_pred)) 


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets